# Data Science Unit 2 Sprint Challenge 3

## Logistic Regression and Beyond

In this sprint challenge you will fit a logistic regression modeling the probability of an adult having an income above 50K. The dataset is available at UCI:

https://archive.ics.uci.edu/ml/datasets/adult

Your goal is to:

1. Load, validate, and clean/prepare the data.
2. Fit a logistic regression model
3. Answer questions based on the results (as well as a few extra questions about the other modules)

Don't let the perfect be the enemy of the good! Manage your time, and make sure to get to all parts. If you get stuck wrestling with the data, simplify it (if necessary, drop features or rows) so you're able to move on. If you have time at the end, you can go back and try to fix/improve.

### Hints

It has a variety of features - some are continuous, but many are categorical. You may find [pandas.get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) (a method to one-hot encode) helpful!

The features have dramatically different ranges. You may find [sklearn.preprocessing.minmax_scale](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.minmax_scale.html#sklearn.preprocessing.minmax_scale) helpful!

## Part 1 - Load, validate, and prepare data

The data is available at: https://archive.ics.uci.edu/ml/datasets/adult

Load it, name the columns, and make sure that you've loaded the data successfully. Note that missing values for categorical variables can essentially be considered another category ("unknown"), and may not need to be dropped.

You should also prepare the data for logistic regression - one-hot encode categorical features as appropriate.

In [2]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
#https://archive.ics.uci.edu/ml/machine-learning-databases/adult/
#https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data

--2019-01-25 11:11:00--  https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data
Resolving archive.ics.uci.edu... 128.195.10.249
Connecting to archive.ics.uci.edu|128.195.10.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3974305 (3.8M) [text/plain]
Saving to: ‘adult.data’

adult.data          100%[===================>]   3.79M  1.11MB/s    in 3.4s    

2019-01-25 11:11:04 (1.11 MB/s) - ‘adult.data’ saved [3974305/3974305]



In [3]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names'

--2019-01-25 11:11:52--  https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names
Resolving archive.ics.uci.edu... 128.195.10.249
Connecting to archive.ics.uci.edu|128.195.10.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5229 (5.1K) [text/plain]
Saving to: ‘adult.names’

adult.names         100%[===================>]   5.11K  --.-KB/s    in 0s      

2019-01-25 11:11:53 (193 MB/s) - ‘adult.names’ saved [5229/5229]



In [4]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test'

--2019-01-25 11:12:25--  https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test
Resolving archive.ics.uci.edu... 128.195.10.249
Connecting to archive.ics.uci.edu|128.195.10.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2003153 (1.9M) [text/plain]
Saving to: ‘adult.test’

adult.test          100%[===================>]   1.91M   363KB/s    in 5.8s    

2019-01-25 11:12:31 (340 KB/s) - ‘adult.test’ saved [2003153/2003153]



In [5]:
!head adult.data

39, State-gov, 77516, Bachelors, 13, Never-married, Adm-clerical, Not-in-family, White, Male, 2174, 0, 40, United-States, <=50K
50, Self-emp-not-inc, 83311, Bachelors, 13, Married-civ-spouse, Exec-managerial, Husband, White, Male, 0, 0, 13, United-States, <=50K
38, Private, 215646, HS-grad, 9, Divorced, Handlers-cleaners, Not-in-family, White, Male, 0, 0, 40, United-States, <=50K
53, Private, 234721, 11th, 7, Married-civ-spouse, Handlers-cleaners, Husband, Black, Male, 0, 0, 40, United-States, <=50K
28, Private, 338409, Bachelors, 13, Married-civ-spouse, Prof-specialty, Wife, Black, Female, 0, 0, 40, Cuba, <=50K
37, Private, 284582, Masters, 14, Married-civ-spouse, Exec-managerial, Wife, White, Female, 0, 0, 40, United-States, <=50K
49, Private, 160187, 9th, 5, Married-spouse-absent, Other-service, Not-in-family, Black, Female, 0, 0, 16, Jamaica, <=50K
52, Self-emp-not-inc, 209642, HS-grad, 9, Married-civ-spouse, Exec-managerial, Husband, White, Male, 0, 0, 45, United-States, >50K
31, 

In [7]:
!cat adult.names

| This data was extracted from the census bureau database found at
| http://www.census.gov/ftp/pub/DES/www/welcome.html
| Donor: Ronny Kohavi and Barry Becker,
|        Data Mining and Visualization
|        Silicon Graphics.
|        e-mail: ronnyk@sgi.com for questions.
| Split into train-test using MLC++ GenCVFiles (2/3, 1/3 random).
| 48842 instances, mix of continuous and discrete    (train=32561, test=16281)
| 45222 if instances with unknown values are removed (train=30162, test=15060)
| Duplicate or conflicting instances : 6
| Class probabilities for adult.all file
| Probability for the label '>50K'  : 23.93% / 24.78% (without unknowns)
| Probability for the label '<=50K' : 76.07% / 75.22% (without unknowns)
|
| Extraction was done by Barry Becker from the 1994 Census database.  A set of
|   reasonably clean records was extracted using the following conditions:
|   ((AAGE>16) && (AGI>100) && (AFNLWGT>1)&& (HRSWK>0))
|
| Prediction task is to determine whether a person makes over

In [2]:
names = ['age',#: continuous.
'workclass',#: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
'fnlwgt',# : continuous.
'education',# : Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
'education-num',#: continuous.
'marital-status',#: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
'occupation',#: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
'relationship',#: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
'race',#: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
'sex',#: Female, Male.
'capital-gain',#: continuous.
'capital-loss',#: continuous.
'hours-per-week',#: continuous.
'native-country',
#: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands
'futureSalaryClass'         
        ]
print(names)


['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'futureSalaryClass']


## Part 2 - Fit and present a Logistic Regression

Your data should now be in a state to fit a logistic regression. Use scikit-learn, define your `X` (independent variable) and `y`, and fit a model.

Then, present results - display coefficients in as interpretible a way as you can (hint - scaling the numeric features will help, as it will at least make coefficients more comparable to each other). If you find it helpful for interpretation, you can also generate predictions for cases (like our 5 year old rich kid on the Titanic) or make visualizations - but the goal is your exploration to be able to answer the question, not any particular plot (i.e. don't worry about polishing it).

It is *optional* to use `train_test_split` or validate your model more generally - that is not the core focus for this week. So, it is suggested you focus on fitting a model first, and if you have time at the end you can do further validation.

In [39]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from datetime import datetime
import math
from scipy.stats import mode
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import statsmodels.api as s
import time
from functools import reduce
import regex
from numpy import array
from numpy import argmax,argmin
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import scale, LabelEncoder, OneHotEncoder
import copy
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm


In [4]:
pd.set_option('display.max_columns', None)
train_df = pd.read_csv('adult.data',header=None, names=names, index_col=False)
train_df.head()
short_df = train_df.sample(frac = 0.01, random_state=42)

In [5]:
short_df.shape

(326, 15)

In [6]:
short_df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,futureSalaryClass
14160,27,Private,160178,Some-college,10,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,38,United-States,<=50K
27048,45,State-gov,50567,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
28868,29,Private,185908,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,Black,Male,0,0,55,United-States,>50K
5667,30,Private,190040,Bachelors,13,Never-married,Machine-op-inspct,Not-in-family,White,Female,0,0,40,United-States,<=50K
7827,29,Self-emp-not-inc,189346,Some-college,10,Divorced,Craft-repair,Not-in-family,White,Male,2202,0,50,United-States,<=50K


In [7]:
# onehot_encoder = OneHotEncoder()
# lab_enc = LabelEncoder()
# def one_hot(df, c, nn):
#   values = array(df[c].values)
#   try:
#        onehot_encoded = onehot_encoder.fit_transform(values)
#   except:
#        values = lab_enc.fit_transform(values)
#        # values = values.ravel()
#        onehot_encoded = onehot_encoder.fit_transform(values)
#   return pd.DataFrame([pd.Series([row[1]] if nn else row) for row in onehot_encoded])

# columns = short_df.select_dtypes(include=[object]).columns
# #columns = ['workclass']
# df = short_df.copy()
# for c in columns:
#    unique = df[c].unique().tolist()
#    print(unique)
#    nn = False
#    if len(unique) == 2:
#       unique = [unique[1]]
#       nn = True
#    df[[(c + "_" + v) for v in unique]] = one_hot(df, c, nn)   
#    df.drop(c, inplace=True, axis=1)
# df.head()

In [8]:
def get_one_hot(cat_types, row, index):
  cell = row.iloc[index]
  pattern = f'{(2 ** (cat_types.index(cell))):016b}' if str(type(cell)) == "<class 'str'>" else '0000000000000000'
  return pd.Series([int(pattern[15-j]) for j in range(0,len(cat_types))])

# bdf = short_df.copy()
# types = bdf.workclass.unique().tolist()
# index = bdf.columns.values.tolist().index('workclass')
# bdf[[('Workclass_' + gtype) for gtype in types]] = \columns = short_df.select_dtypes(include=[object]).columns
# bdf[[('garageType_' + gtype) for gtype in types]] = \
#        bdf.apply(lambda row: get_one_hot(types, row, index), axis=1) 
df = short_df.copy()
columns = df.select_dtypes(include=[object]).columns.drop(['sex', 'futureSalaryClass'])
# columns = ['workclass']
for c in columns:
   unique = df[c].unique().tolist()
   types = df[c].unique().tolist()
   index = df.columns.values.tolist().index(c) 
   #print(unique)
   nn = False
   if len(unique) == 2:
      unique = [unique[1]]
      nn = True
   df[[(c + "_" + v) for v in unique]] =  df.apply(lambda row: get_one_hot(types, row, index), axis=1)   
   df.drop(c, inplace=True, axis=1)
df.head()


,age,fnlwgt,education-num,sex,capital-gain,capital-loss,hours-per-week,futureSalaryClass,workclass_ Private,workclass_ State-gov,workclass_ Self-emp-not-inc,workclass_ Self-emp-inc,workclass_ Federal-gov,workclass_ Local-gov,workclass_ ?,education_ Some-college,education_ HS-grad,education_ Bachelors,education_ Masters,education_ Assoc-acdm,education_ 7th-8th,education_ 11th,education_ Assoc-voc,education_ Prof-school,education_ 9th,education_ 5th-6th,education_ 10th,education_ Doctorate,education_ 12th,marital-status_ Divorced,marital-status_ Married-civ-spouse,marital-status_ Never-married,marital-status_ Separated,marital-status_ Widowed,marital-status_ Married-spouse-absent,occupation_ Adm-clerical,occupation_ Exec-managerial,occupation_ Machine-op-inspct,occupation_ Craft-repair,occupation_ Prof-specialty,occupation_ Sales,occupation_ Handlers-cleaners,occupation_ Other-service,occupation_ Protective-serv,occupation_ Priv-house-serv,occupation_ Transport-moving,occupation_ ?,occupation_ Farming-fishing,occupation_ Tech-support,relationship_ Not-in-family,relationship_ Wife,relationship_ Husband,relationship_ Unmarried,relationship_ Own-child,relationship_ Other-relative,race_ White,race_ Black,race_ Other,race_ Asian-Pac-Islander,race_ Amer-Indian-Eskimo,native-country_ United-States,native-country_ Mexico,native-country_ Puerto-Rico,native-country_ El-Salvador,native-country_ ?,native-country_ Columbia,native-country_ Poland,native-country_ Cambodia,native-country_ Germany,native-country_ Philippines,native-country_ Canada,native-country_ Ecuador,native-country_ Italy,native-country_ Cuba,native-country_ Iran
14160,27,160178,10,Female,0,0,38,<=50K,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
27048,45,50567,9,Female,0,0,40,<=50K,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
28868,29,185908,13,Male,0,0,55,>50K,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5667,30,190040,13,Female,0,0,40,<=50K,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7827,29,189346,10,Male,2202,0,50,<=50K,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
df.sex.unique()
df['Female'] = df.sex.map({' Male': 0, ' Female': 1})


In [10]:
df.futureSalaryClass.unique()
target = df.futureSalaryClass.map({' <=50K':1,' >50K': 0 })

In [11]:
dff = df.drop(['sex', 'futureSalaryClass'], axis=1)

In [12]:
dff.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_ Private,workclass_ State-gov,workclass_ Self-emp-not-inc,workclass_ Self-emp-inc,workclass_ Federal-gov,workclass_ Local-gov,workclass_ ?,education_ Some-college,education_ HS-grad,education_ Bachelors,education_ Masters,education_ Assoc-acdm,education_ 7th-8th,education_ 11th,education_ Assoc-voc,education_ Prof-school,education_ 9th,education_ 5th-6th,education_ 10th,education_ Doctorate,education_ 12th,marital-status_ Divorced,marital-status_ Married-civ-spouse,marital-status_ Never-married,marital-status_ Separated,marital-status_ Widowed,marital-status_ Married-spouse-absent,occupation_ Adm-clerical,occupation_ Exec-managerial,occupation_ Machine-op-inspct,occupation_ Craft-repair,occupation_ Prof-specialty,occupation_ Sales,occupation_ Handlers-cleaners,occupation_ Other-service,occupation_ Protective-serv,occupation_ Priv-house-serv,occupation_ Transport-moving,occupation_ ?,occupation_ Farming-fishing,occupation_ Tech-support,relationship_ Not-in-family,relationship_ Wife,relationship_ Husband,relationship_ Unmarried,relationship_ Own-child,relationship_ Other-relative,race_ White,race_ Black,race_ Other,race_ Asian-Pac-Islander,race_ Amer-Indian-Eskimo,native-country_ United-States,native-country_ Mexico,native-country_ Puerto-Rico,native-country_ El-Salvador,native-country_ ?,native-country_ Columbia,native-country_ Poland,native-country_ Cambodia,native-country_ Germany,native-country_ Philippines,native-country_ Canada,native-country_ Ecuador,native-country_ Italy,native-country_ Cuba,native-country_ Iran,Female
14160,27,160178,10,0,0,38,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
27048,45,50567,9,0,0,40,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
28868,29,185908,13,0,0,55,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5667,30,190040,13,0,0,40,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
7827,29,189346,10,2202,0,50,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
columns = dff.columns
df_s = pd.DataFrame(scale(dff), columns=columns)

In [15]:
X = df_s
y = target
log_reg = LogisticRegression(solver='lbfgs', max_iter=1000, multi_class='multinomial')
log_reg.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

## Part 3 - Analysis, Interpretation, and Questions

### Based on your above model, answer the following questions

1. What are 3 features positively correlated with income above 50k?
2. What are 3 features negatively correlated with income above 50k?
3. Overall, how well does the model explain the data and what insights do you derive from it?

*These answers count* - that is, make sure to spend some time on them, connecting to your analysis above. There is no single right answer, but as long as you support your reasoning with evidence you are on the right track.

Note - scikit-learn logistic regression does *not* automatically perform a hypothesis test on coefficients. That is OK - if you scale the data they are more comparable in weight.

### Match the following situation descriptions with the model most appropriate to addressing them

In addition to logistic regression, a number of other approaches were covered this week. Pair them with the situations they are most appropriate for, and briefly explain why.

Situations:
1. You are given data on academic performance of primary school students, and asked to fit a model to help predict "at-risk" students who are likely to receive the bottom tier of grades.
2. You are studying tech companies and their patterns in releasing new products, and would like to be able to model and predict when a new product is likely to be launched.
3. You are working on modeling expected plant size and yield with a laboratory that is able to capture fantastically detailed physical data about plants, but only of a few dozen plants at a time.

Approaches:
1. Ridge Regression
2. Quantile Regression
3. Survival Analysis

In [19]:
log_reg.score(X, y)

0.8926380368098159

In [37]:
o = []
for x,c in zip(log_reg.coef_[0],columns):
   o.append({'v': x, 'c': c})

#list1.sort(key = sortSecond, reverse = True) 
o.sort(key = lambda d: d['v'], reverse = True)
print(o[0:3])
o.sort(key = lambda d: d['v'], reverse = False)
print('____________________\n',o[0:3])

[{'v': 0.34614639745440307, 'c': 'occupation_ Tech-support'}, {'v': 0.29489245891207849, 'c': 'relationship_ Own-child'}, {'v': 0.26965387259245555, 'c': 'education_ 11th'}]
____________________
 [{'v': -1.2715625581850796, 'c': 'capital-gain'}, {'v': -0.43495010377705873, 'c': 'hours-per-week'}, {'v': -0.34339799468005605, 'c': 'marital-status_ Married-civ-spouse'}]


Positive features: 'capitol-gain', 'hours-per-week', 'marital-status == 'Married-civ-spouse'
Negative features: 'occupation' == 'Tech-support', 'relationship' == 'Own-child', 'education' == '11th'

Having large Capitol Gains and working long hours is good for future income, so is being married.
Working a crummy job, having a child to support, and not graduating high school are bad for future earnings.

In [43]:
short_df['marital-status'].unique()

array([' Divorced', ' Married-civ-spouse', ' Never-married', ' Separated',
       ' Widowed', ' Married-spouse-absent'], dtype=object)

In [44]:
short_df.relationship.unique()

array([' Not-in-family', ' Wife', ' Husband', ' Unmarried', ' Own-child',
       ' Other-relative'], dtype=object)

In [41]:
X = sm.add_constant(X)

model = sm.OLS(list(target), X).fit()
predictions_test = model.predict(X) 

print_model_test = model.summary()
print(print_model_test)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.510
Model:                            OLS   Adj. R-squared:                  0.393
Method:                 Least Squares   F-statistic:                     4.334
Date:                Fri, 25 Jan 2019   Prob (F-statistic):           2.92e-17
Time:                        13:47:12   Log-Likelihood:                -76.586
No. Observations:                 326   AIC:                             281.2
Df Residuals:                     262   BIC:                             523.5
Df Model:                          63                                         
Covariance Type:            nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------

**TODO - your answers!**